# Rendering Animations of HDM5 Dataset in Blender

## Setup

1. Get the raw files
2. Open this tutorial

In [1]:
import os
os.chdir("../../../../../src")

import bpy
from tqdm import tqdm
from dataset.hdm05_params import ASF_JOINT2DOF
from render.blender.utils import *
import numpy as np


In [2]:
# Specify which amc file and fbx file to render
amc_file_path = "../../../../Downloads/HDM_tr_01-01_01_120.amc"
fbx_file_path = "../dataset/HDM05/skeleton/tr.fbx"

In [3]:
import_fbx(fbx_file_path)

FBX Import: start importing ../dataset/HDM05/skeleton/tr.fbx
FBX version: 7700
	FBX import: Prepare...
		Done (0.000625 sec)

	FBX import: Templates...
		Done (0.000040 sec)

	FBX import: Nodes...
		Done (0.000766 sec)

	FBX import: Connections...
		Done (0.000516 sec)

	FBX import: Meshes...
		Done (0.000424 sec)

	FBX import: Materials & Textures...
		Done (0.000062 sec)

	FBX import: Cameras & Lamps...
		Done (0.000061 sec)

	FBX import: Objects & Armatures...
		Done (0.007301 sec)

	FBX import: ShapeKeys...
		Done (0.000081 sec)

	FBX import: Animations...
		Done (0.000100 sec)

	FBX import: Assign materials...
		Done (0.000033 sec)

	FBX import: Assign textures...
		Done (0.000728 sec)

	FBX import: Cycles z-offset workaround...
		Done (0.000408 sec)

	Done (0.016296 sec)

Import finished.


In [24]:
with open(amc_file_path, "rb") as f:
    cur_frame = 0
    character = bpy.data.objects["Armature"]
    character.select_set(True)
    bpy.ops.object.mode_set(mode="POSE")
    for line in tqdm(f.readlines()):
        if line.strip().isdigit():
            cur_frame = int(line)
        elif cur_frame > 0 and cur_frame % 100 == 0:
            data = line.decode("utf-8").strip().split()
            joint_name = data[0]
            joint = character.pose.bones[joint_name]
            character.pose.bones[joint_name].rotation_mode = "XYZ"
            if joint_name == "root":
                set_joint_location_keyframe(joint, np.float_(data[1:4]), cur_frame)
                set_joint_rotation_keyframe(joint, np.float_(data[4:]) * np.pi / 180, cur_frame)
            else:
                set_joint_rotation_keyframe(joint, np.float_(data[1:]) * np.pi / 180, cur_frame, axis=ASF_JOINT2DOF[joint_name])
                
    bpy.data.scenes["Scene"].frame_start = 1
    bpy.data.scenes["Scene"].frame_end = cur_frame

100%|████████████████████████████████████████| 266793/266793 [00:00<00:00, 1743368.87it/s]


In [23]:
clear_all_animation()